Pada tugas pratikum ini Anda akan menggunakan data "Wisconsin Breast Cancer". Data tersebut terdiri dari 569 data yang digunakan untuk mendiagnonis jenis kanker Malignant (M) dan Benign (B). Tugas Anda adalah,
1. Pisahkan antara variabel yang dapat digunakan dan variabel yang tidak dapat digunakan.
2. Lakukan proses encoding pada kolom "diagnosis".
3. Lakukan proses standardisasi pada semua kolom yang memiliki nilai numerik.

In [15]:
from polars import DataFrame, Float64, Series
from typing import Tuple
import polars as pl

In [16]:
def load_and_process_wbc(csv_path: str) -> Tuple[DataFrame, Series]:
    # 1. Baca CSV
    df: DataFrame = pl.read_csv(csv_path)

    # 2a. Pisahkan fitur dan label
    features: DataFrame = df.drop(['id', 'diagnosis'])

    # 2b. Cast semua kolom numerik ke float
    features = features.select([pl.col(c).cast(Float64) for c in features.columns])

    # 3. Encoding kolom diagnosis: M -> 1, B -> 0
    label_encoded: Series = (
        df.select(pl.when(pl.col("diagnosis") == "M").then(1).otherwise(0).alias("diagnosis"))
        .to_series()
        .cast(pl.Int8)
    )

    # 4. Standardisasi fitur numerik sepenuhnya di Polars
    features_scaled: DataFrame = features.select([
        ((pl.col(c) - pl.col(c).mean()) / pl.col(c).std()).alias(c)
        for c in features.columns
    ])

    return features_scaled, label_encoded


# --- Eksekusi ---
features_scaled, label_encoded = load_and_process_wbc("wbc.csv")

print("Kolom fitur setelah standardisasi:")
print(features_scaled.columns)

print("\nContoh data fitur setelah standardisasi:")
print(features_scaled.head())

print("\nContoh label encoded:")
print(label_encoded.head())

Kolom fitur setelah standardisasi:
['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean', 'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se', 'fractal_dimension_se', 'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst', 'concave points_worst', 'symmetry_worst', 'fractal_dimension_worst', '']

Contoh data fitur setelah standardisasi:
shape: (5, 31)
┌────────────┬────────────┬────────────┬───────────┬───┬────────────┬───────────┬───────────┬──────┐
│ radius_mea ┆ texture_me ┆ perimeter_ ┆ area_mean ┆ … ┆ concave    ┆ symmetry_ ┆ fractal_d ┆      │
│ n          ┆ an         ┆ mean       ┆ ---       ┆   ┆ points_wor ┆ worst     ┆ imension_ ┆ ---  │
│ ---        ┆ ---        ┆ ---        ┆ f64       ┆   